In [ ]:
%pylab inline
from imp import reload
import scipy.stats as stats
import shardlib.trees as tr
import shardlib.plotFuncs as pf
import shardlib.dbaccess as dba
import shardlib.dataProcessing as dp
import shardlib.sessionFuncs as sf
import shardlib.queues as qs
from shardlib.comp_analysis import XCPCompAnalysis
from shardlib.comp_analysis import XSCompAnalysis
from shardlib.comp_analysis import FEMCompAnalysis
from shardlib.comp_analysis import SIMCompAnalysis
from shardlib.comp_analysis import PSCCompAnalysis

[Documentation](docs/enrupture_guide.pdf)

# Contents

<b>[1. Initialization](#1.-Initialization)</b>

<a href="#1(a)-New-Session">1(a) New Session</a>

 <a href="#1(b)-Number-and-Distribution-of-the-Simulations">1(b) Number and Distribution of the Simulations</a>

 <a href="#1(c)-Container-size-convergence">1(c) Container size convergence</a>

 <a href="#1(d)-Number-and-Distribution-of-Simulations-with-Optimal-Size-Container">1(d) Number and Distribution of Simulations with Optimal Size Container</a>

<b><a href="#2.-Analysis-of-the-Model-Types">2. Analysis of the Model Types</a></b>

 <i><a href="#2(a)-XFEM-crackPartition">2(a) XFEM crackPartition</a></i>

  <a href="#2(a)i.-Number-and-Distribution-of-XFEM-CP-Simulations">2(a)i. Number and Distribution of XFEM CP Simulations</a>

  <a href="#2(a)ii.-Initialization">2(a)ii. Initialization</a>

  <a href="#2(a)iii.-Analysis-Step">2(a)iii. Analysis Step</a>

 <i><a href="#2(b)-XFEM-simple">2(b) XFEM simple</a></i>

  <a href="#2(b)i.-Number-and-Distribution-of-XFEM-simple-Simulations">2(b)i. Number and Distribution of XFEM simple Simulations</a>

  <a href="#2(b)ii.-Initialization">2(b)ii. Initialization</a>

  <a href="#2(b)iii.-Analysis-step">2(b)iii. Analysis Step</a>

 <i><a href="#2(c)-FEM">2(c) FEM</a></i>

  <a href="#2(c)i.-Container-Size-Comparison-for-XFEM-and-FEM-Analyses">2(c)i. Container Size Comparison for XFEM and FEM Analyses</a>

  <a href="#2(c)ii.-Number-and-Distribution-of-FEM-Simulations">2(c)ii. Number and Distribution of FEM Simulations</a>

  <a href="#2(c)iii.-Initialization">2(c)iii. Initialization</a>

  <a href="#2(c)iv.-Analysis-Step">2(c)iv. Analysis Step</a>

<b><a href="#3.-Comparison-of-the-Model-Types">3. Comparison of the Model Types</a></b>

 <a href="#3(a)-Initialization">3(a) Initialization</a>

 <a href="#3(b)-Analysis-Step">3(b) Analysis Step</a>

# 1. Initialization

## 1(a) New Session

In [ ]:
sf.createNewSessionFiles(False)

## 1(b) Number and Distribution of the Simulations

### 1(b)i. All Simulations

In [ ]:
allSimTree = tr.createTreeFromDbKeys(dba.getAllShelveKeys())
allKeysTree = tr.createTreeOfKeys(allSimTree)
sf.setSimIdAsFailed([allSimTree], sf.getSimIdsFromShelve('failed'))
tr.assignBarWidthsAndMarks(allSimTree, allKeysTree)
tr.assignLegendNames(allSimTree)
allSimTree.printStats2()
fig = figure(figsize=(12.9,8),dpi=100)
tr.barPlot(allSimTree, allKeysTree, fig)

### 1(b)ii. Simulations with Equal Initial Conditions and Material

In [ ]:
dbKeys = pf.filterSimIds(dba.getAllShelveKeys(), 
                         {'sigma':100, 'omega':60, 'gamma':45, 'v':0.3, 'E':2e5})
simTree = tr.createTreeFromDbKeys(dbKeys)
sf.setSimIdAsFailed([allSimTree, simTree], sf.getSimIdsFromShelve('failed'))
keysTree = tr.createTreeOfKeys(simTree)
simTree.printStats2()

In [ ]:
tr.assignBarWidthsAndMarks(simTree, keysTree)
tr.assignLegendNames(simTree)
fig = figure(figsize=(12.9,8),dpi=100)
tr.barPlot(simTree, keysTree, fig)

### 1(b)iii. Assignment of Crack Ratio

In [ ]:
crackRatio = '10.0'

### 1(b)iv. Simulations with Equal Crack Ratio

In [ ]:
crKeys = pf.filterSimIds(dbKeys, {'crackRatio':float(crackRatio)})
crTree = tr.createTreeFromDbKeys(crKeys)
sf.setSimIdAsFailed([allSimTree, simTree, crTree], sf.getSimIdsFromShelve('failed'))
crKeysTree = tr.createTreeOfKeys(crTree)
crTree.printStats2()

In [ ]:
tr.assignBarWidthsAndMarks(crTree, crKeysTree)
tr.assignLegendNames(crTree)
fig = figure(figsize=(12,8),dpi=100)
tr.barPlot(crTree, crKeysTree, fig)

## 1(c) Container size convergence

### 1(c)i. Contour Plots for SCS - Preview Stage

In [ ]:
analysisType = 'XFEM'
modelType = 'crackPartition'
elements = 'LinearTet'
s_sims, f_sims = pf.getBranchSimIdsByCriteria(crTree, [analysisType, modelType, elements])
errDictCSConv = pf.selectSimsForMinErrors(
            s_sims, ['areaDiff', 'avgNormError', 'maxNormError', 'dotProd'], ['K1', 'K2', 'K3'])
for errType in ['areaDiff', 'avgNormError', 'dotProd', 'maxNormError']:
    fig = figure(figsize=(15,5), dpi=100)
    pf.createContourPlot4(errDictCSConv[errType], fig, errType, levels=15)

### 1(c)ii. Optimal Container Size

In [ ]:
optD = 180
optH = 120

### 1(c)iii. Contour Plots for CSC - Verification Stage

In [ ]:
criteriaParam = 'rmsd' #'avgNormError', 'areaDiff', 'dotProd', 'maxNormError'
criteriaSif = 'K1' #'K2', 'K3'
errorTypes = ['areaDiff', 'avgNormError', 'dotProd', 'maxNormError']
SIFs = ['K1', 'K3'] #'K2', 
d = pf.selectSimsForMinErrors(s_sims, [criteriaParam], [criteriaSif])[criteriaParam][criteriaSif]
selSim = set([d[k][0] for k in d.keys()])
d = pf.selectSimsForMinErrors(selSim, errorTypes, SIFs)
for errType in errorTypes:
    fig = figure(figsize=(15,5), dpi=100)
    pf.createContourPlot4(d[errType], fig, errType, selected=(optD,optH), levels=15)

## 1(d) Number and Distribution of Simulations with Optimal Size Container

In [ ]:
dhcrKeys = pf.filterSimIds(crKeys, {'d':optD, 'h':optH})
dhcrTree = tr.createTreeFromDbKeys(dhcrKeys)
sf.setSimIdAsFailed([allSimTree, simTree, crTree, dhcrTree], sf.getSimIdsFromShelve('failed'))
dhcrKeysTree = tr.createTreeOfKeys(dhcrTree)
dhcrTree.printStats2()

In [ ]:
tr.assignBarWidthsAndMarks(dhcrTree, dhcrKeysTree)
tr.assignLegendNames(dhcrTree)
fig = figure(figsize=(12,8),dpi=100)
tr.barPlot(dhcrTree, dhcrKeysTree, fig)

---------------

# 2. Analysis of the Model Types

## 2(a) XFEM crackPartition

### 2(a)i. Number and Distribution of XFEM CP Simulations

#### 2(a)iA. Stats for all XFEM CP Simulations with the Specified Crack Ratio

In [ ]:
crTree.getTreeBranch(['XFEM', 'crackPartition', 'LinearTet']).printStats2()

#### 2(a)iB. Successful Simulations vs Seed Parameters Plots

In [ ]:
trbr_xcp = crTree.getTreeBranch(['XFEM', 'crackPartition', 'LinearTet'])
crackEdges = dba.getParameterRange(trbr_xcp.getAllMembers(), 'crackEdges')
for ce in sorted(crackEdges):
    sf_xcp = pf.filterSimIds(trbr_xcp.getSuccessfulMembers(), {'crackEdges':ce})
    ff_xcp = pf.filterSimIds(trbr_xcp.getFailedMembers(), {'crackEdges':ce})
    fig = figure(figsize=(8.1,5),dpi=100)
    pf.create3dScatterPlotSummary(sf_xcp, ff_xcp, 'Crack edges: {0}'.format(ce), fig)

#### 2(a)iC. Stats for all XFEM CP Simulations with the Specified Crack Ratio and Optimal Container Size

In [ ]:
dhcrTree.getTreeBranch(['XFEM', 'crackPartition', 'LinearTet']).printStats2()

### 2(a)ii. Initialization

In [ ]:
criteria_xcp = ['rmsd', 'K1']
sifs_xcp = ['K1', 'K2', 'K3']

In [ ]:
cq_xcp = qs.CompQueue()

In [ ]:
cq_xcp.loadSession(dhcrTree.getTreeBranch(['XFEM', 'crackPartition', 'LinearTet']))
cq_xcp.printQueue()

### 2(a)iii. Analysis Step

#### 2(a)iiiA. Box Plot

In [ ]:
crackEdge = 0.4
fig = figure(figsize=(12,10), dpi=100)
xcp = XCPCompAnalysis.XCPCompAnalysis(dhcrTree.getTreeBranch(['XFEM', 'crackPartition', 'LinearTet']),
                          crackEdge, criteria_xcp, sifs_xcp)
xcp.runMethods()
xcp.createFigure(fig)

#### 2(a)iiiB. Assignment of Failed simIds

In [ ]:
selectionIdNumber = None
xcp.assignSimIdAsFailed([allSimTree, simTree, crTree, dhcrTree], cq_xcp, selectionIdNumber)

#### 2(a)iiiC. Add simIds to CQ

In [ ]:
selectionIdNumber = 3
xcp.addToQueue(cq_xcp, selectionIdNumber)
cq_xcp.printQueue()

#### 2(a)iiiD. SIFs Plots of simIds in the CQ

In [ ]:
fig = figure(figsize(8,10), dpi=100)
cq_xcp.plotComp(fig)
cq_xcp.printQueue()

#### 2(a)iiiE. Remove simId from CQ

In [ ]:
num = None
cq_xcp.removeFromQueue(num)
cq_xcp.printQueue()

#### 2(a)iiiF. Errors and SIFs Plots for a Selected simId

In [ ]:
queue_num_xcp = 1
cq_xcp.printQueue()
fig = figure(figsize=(8,10), dpi=100)
pf.plotSimId(cq_xcp.getByNumber(queue_num_xcp), fig)

#### 2(a)iiiG. Save a Selected simId to the Session successful File

In [ ]:
sf.writeToShelve(cq_xcp.getByNumber(queue_num_xcp), 'successful')

------------

## 2(b) XFEM simple

### 2(b)i. Number and Distribution of XFEM simple Simulations

#### 2(b)iA. Stats for All XFEM simple Simulations with the Specified Crack Ratio

In [ ]:
crTree.getTreeBranch(['XFEM', 'simple']).printStats2()

#### 2(b)iB. Successful Simulations vs Seed Parameters Plots

In [ ]:
for n in sorted(crTree.getTreeBranch(['XFEM', 'simple']).getChildren(), key=lambda x: x.getName()):
    fig = figure(figsize=(8.1,5),dpi=100)
    title = 'Elements - {0}'.format(n.getName())
    pf.create3dScatterPlotSummary(n.getSuccessfulMembers(), n.getFailedMembers(), title, fig)

#### 2(b)iC. Stats for All XFEM simple Simulations with the Specified Crack Ratio and Optimal Container Size

In [ ]:
dhcrTree.getTreeBranch(['XFEM', 'simple']).printStats2()

### 2(b)ii. Initialization

In [ ]:
criteria_xs = ['rmsd', 'K1']
sifs_xs = ['K1', 'K2', 'K3']

In [ ]:
cq_xs = qs.CompQueue()

In [ ]:
elements_xs = ['LinearHexRI', 'LinearHexFI', 'LinearTet'][2]
cq_xs.loadSession(dhcrTree.getTreeBranch(['XFEM', 'simple', elements_xs]))
cq_xs.printQueue()

### 2(b)iii. Analysis step

#### 2(b)iiiA. Box Plots

In [ ]:
analysis1 = ['LinearHexRI', 'LinearHexFI', 'LinearTet'][0]
analysis2 = ['LinearHexRI', 'LinearHexFI', 'LinearTet'][2]
xs = XSCompAnalysis.XSCompAnalysis(pNode = dhcrTree,
    analysisBranches = [['XFEM', 'simple', analysis1], ['XFEM', 'simple', analysis2]], 
    criteria = criteria_xs, sifs = sifs_xs)
fig = figure(figsize=(12,10), dpi=100) #10
xs.runMethods()
xs.createFigure(fig)

#### 2(b)iiiB. Assignment of Failed simIds

In [ ]:
selectionIdNumber = 5
xs.assignSimIdAsFailed([allSimTree, simTree, crTree, dhcrTree], cq_xs, selectionIdNumber)

#### 2(b)iiiC. Add simIds to CQ

In [ ]:
selectionIdNumber = 4
xs.addToQueue(cq_xs, selectionIdNumber)
cq_xs.printQueue()

#### 2(b)iiiD. SIFs Plots of simIds in CQ

In [ ]:
fig = figure(figsize(8,10), dpi=100)
cq_xs.plotComp(fig)
cq_xs.printQueue()

#### 2(b)iiiE. Remove a simId from CQ

In [ ]:
num = None
cq_xs.removeFromQueue(num)
cq_xs.printQueue()

#### 2(b)iiiF. Errors and SIFs Plots for a Selected simId

In [ ]:
queue_num_xs = 1
cq_xs.printQueue()
fig = figure(figsize=(8,10), dpi=100)
pf.plotSimId(cq_xs.getByNumber(queue_num_xs), fig)

#### 2(b)iiiG. Save a Selected simId to the Session successful File

In [ ]:
sf.writeToShelve(cq_xs.getByNumber(queue_num_xs), 'successful')

---------------

## 2(c) FEM

### 2(c)i. Container Size Comparison for XFEM and FEM Analyses

In [ ]:
fig = figure(figsize=(8,10), dpi=100)
pf.plotContainerSizeComparison(optD, optH, crackRatio, simTree, fig)

### 2(c)ii. Number and Distribution of FEM Simulations

#### 2(c)iiA. Stats for All FEM Simulations with the Specified Crack Ratio

In [ ]:
crTree.getTreeBranch(['FEM']).printStats2()

#### 2(c)iiB. Stats for All FEM Simulations with the Specified Crack Ratio and Optimal Container Size

In [ ]:
dhcrTree.getTreeBranch(['FEM']).printStats2()

### 2(c)iii. Initialization

In [ ]:
criteria_fc = ['rmsd', 'K1']
sifs_fc = ['K1', 'K2', 'K3']

In [ ]:
cq_fc = qs.CompQueue()

In [ ]:
cq_fc.loadSession(dhcrTree.getTreeBranch(['FEM'])) # 'elliptic', 'LinearHexRI'
cq_fc.printQueue()

### 2(c)iv. Analysis Step

#### 2(c)ivA. Box Plot

In [ ]:
analysis1 = ['FEM', 'elliptic', 'QuadraticRI']
analysis2 = ['FEM', 'simpleScale', 'LinearRI']
fc = FEMCompAnalysis.FEMCompAnalysis(dhcrTree, [analysis1, analysis2], criteria_fc, sifs_fc)
fig = figure(figsize=(12,10), dpi=100) #10
fc.runMethods()
fc.createFigure(fig)

#### 2(c)ivB. Assignment of Failed simIds

In [ ]:
selectionIdNumber = None
fc.assignSimIdAsFailed([allSimTree, simTree, crTree, dhcrTree], cq_fc, selectionIdNumber)

#### 2(c)ivC. Add simIds to CQ

In [ ]:
selectionIdNumber = 1
fc.addToQueue(cq_fc, selectionIdNumber)
cq_fc.printQueue()

#### 2(c)ivD. SIFs Plots of simIds in CQ

In [ ]:
fig = figure(figsize(8,10), dpi=100)
cq_fc.plotComp(fig)

#### 2(c)ivE. Remove a simId from the CQ

In [ ]:
num = None
cq_fc.removeFromQueue(num)
cq_fc.printQueue()

#### 2(c)ivF. Errors and SIFs Plots for a Selected simId

In [ ]:
queue_num_fc = 1
fig = figure(figsize=(8,10), dpi=100)
pf.plotSimId(cq_fc.getByNumber(queue_num_fc), fig)

#### 2(c)ivG. Save a Selected simId to the Session successful File

In [ ]:
sf.writeToShelve(cq_fc.getByNumber(queue_num_fc), 'successful')

---------------

# 3. Comparison of the Model Types

## 3(a) Initialization

In [ ]:
criteria_sc = ['rmsd', 'K1']
sifs_sc = ['K1', 'K2', 'K3']

In [ ]:
lq = qs.LeavesQueue()
for i in tr.getTreeLeaves(dhcrTree):
    lq.addLeaf(i)
lq.printQueue()

## 3(b) Analysis Step

#### 3(b)i. Box Plot

In [ ]:
sc = SIMCompAnalysis.BoxCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(12,8),dpi=100)
sc.createCompBoxPlot([3,9,10,11,12], 'difference', fig)

#### 3(b)ii. Histogram

In [ ]:
sc = SIMCompAnalysis.HistCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(10,10),dpi=100)
items = {1:20, 2:20, 9:200}
sc.createCompHistPlot(items, 'difference', [-50, 50], fig)

#### 3(b)iii. Correlation Plot

In [ ]:
sc = SIMCompAnalysis.CorrCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(10,10),dpi=100)
sc.createCompCorrPlot([1,2,3,9], 'difference', [-100, 100], fig)

In [ ]:
sc = SIMCompAnalysis.CorrCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(10,10),dpi=100)
sc.createCompCorrPlot([1, 9], 'results', None, fig)

#### 3(b)iv. Range Plot

In [ ]:
sc = SIMCompAnalysis.RangeCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(12,14),dpi=100)
opts = {'data':['results', 'errors'],'errors':'difference', 'optSim':True, 'range':True,
        'analytical':True, 'dataPoints':True, 'rangeType':1}
sc.createCompRangePlot({3:11,12:14}, opts, fig)  # 1:7, 2:7, 3:11, 4:11, 5:7, 6:7, 7:11, 8:11, 9:79, 10:29, 11:29, 12:14

#### 3(b)v. Bounds Plot

In [ ]:
sc = SIMCompAnalysis.BoundsCompPlot(lq, criteria_sc, sifs_sc)
fig = figure(figsize=(12,14),dpi=100)
targets = {'lower':5, 'upper':95}
sc.createBoundsPlot([9, 10], targets, fig, tol=0.1, iterLim=100)

-----------------

# One more thing...

In [ ]:
allSimTree.getTreeBranch([1.0]).printStats2()

In [ ]:
branch = [1.0, 'XFEM', 'crackPartition', 'LinearTet']
filterDict = {'sigma':100, 'omega':0, 'gamma':0, 'v':0.3, 'E':2e5,
              'crackEdges':0.05, 'allEdges':2}
cq_psc = qs.CompQueue()
node = allSimTree.getTreeBranch(branch)
cq_psc.addSimId(pf.filterSimIds(node.getSuccessfulMembers(), filterDict))
#cq_psc.loadSession(allSimTree.getTreeBranch(branch))
print len(cq_psc.getQueue())

In [ ]:
psc = PSCCompAnalysis.PSCCompAnalysis(cq_psc.getQueue())
fig = figure(figsize=(12,10), dpi=100)
psc.createPSCStatPlots(100, 120, 50, 50, fig)

In [ ]:
fig = figure(figsize=(12,10), dpi=100)
psc.createPSCProbPlots(2, fig)

In [ ]:
fig = figure(figsize=(12,10), dpi=100)
cq_psc.plotComp(fig)

In [ ]:
fig = figure(figsize=(10,10), dpi=100)
pf.plotSimId(cq_psc.getByNumber(19), fig)

In [ ]:
PSCCompAnalysis.assignSimIdAsFailed([allSimTree, simTree, crTree, dhcrTree], cq_psc, 19) #, 

In [ ]:
#cq_psc.removeFromQueue(16)
cq_psc.printQueue()